In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path
scripts_path = Path("../Data-Preprocessing/").resolve()
sys.path.append(str(scripts_path))

In [3]:
import pickle
from sklearn.cluster import KMeans
from scripts.data_visualiser import *
from scripts.data_loader import *
from scripts.data_preprocessor import *
from scripts.temporal_data_preprocessor import *
from scripts.temporal_data_loader import *
from scripts.temporal_visualiser import *
from scripts.temporal_chanel_refinement import *
from model_scripts.feature_extraction import *
from model_scripts.get_statistics import *
from Pipeline.pre_processing_pipeline import *
import numpy as np
import preprocessing_config as config

import torch
import torch.nn as nn
from torchvision.models import resnet50

In [4]:
temporal_images = load_field_images_temporal(config.base_directory_temporal)

In [5]:
len(temporal_images)

48

### Resnet-50 for Feature Extraction - RGB + 3 bands

Resnet-50: Pretrained on the ImageNet dataset

In [6]:
simple_images = get_non_temporal_images(temporal_images)

In [21]:
simple_images[0].shape

(64, 64, 13)

In [22]:
refined_images, field_numbers = refine_chanel_non_temporal(simple_images)

In [23]:
refined_images[0].shape

(64, 64, 6)

In [41]:
resized_images = resize_with_padding(refined_images)
resized_images[0].shape

(224, 224, 6)

#### Model

In [43]:
# Load pretrained ResNet model
model = resnet50(pretrained=True)

# Modify the first convolutional layer to accept 6 channels
model.conv1 = nn.Conv2d(6, 64, kernel_size=7, stride=2, padding=3, bias=False)

# Remove the final fully connected layer
feature_extractor = nn.Sequential(*list(model.children())[:-1])

/Users/bhumikasadbhave007/tensorflow-test/env/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/bhumikasadbhave007/tensorflow-test/env/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [44]:
images_tensors = [torch.from_numpy(img).float() for img in resized_images]
stacked_images = torch.stack(images_tensors, dim=0)
stacked_images.shape

torch.Size([48, 224, 224, 6])

In [45]:
features = extract_features_cnn(stacked_images, feature_extractor)
print("Features Shape:", features.shape)  # (n, 2048)

/Users/bhumikasadbhave007/Documents/THWS/Semester-4/MASTER-THESIS/Master-Thesis-Github/Master-Thesis/Modeling/model_scripts/feature_extraction.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


Features Shape: (48, 2048)


In [46]:
n_clusters = 2  
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(features)

labels = kmeans.labels_

/Users/bhumikasadbhave007/tensorflow-test/env/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [47]:
acc = get_accuracy(field_numbers, labels, config.labels_path)
acc

0.5081967213114754

### Resnet-50 for Feature Extraction - VIs
- All Bands = 55.73%
- NDVI, MSI, CI, EVI = 47.54%
- NDVI, MSI, CI, EVI + Bands = 52.45%
- MSI + Bands = 57.37%
- NDVI + Bands = 45.90%
- EVI + MSI = 47.54%
- EVI + MSI + Bands = 50.81%
- Bands only 0,1,2,3,8,9 = 59.01%

In [241]:
temporal_images[0][0].shape

(64, 64, 13)

In [260]:
field_numbers, indices_images = indexbands_temporal_cubes(temporal_images, 'msi')
#indices_images = np.expand_dims(indices_images, axis=-1)
indices_images[0][0].shape

(64, 64, 3)

In [261]:
simple_images = get_non_temporal_images(indices_images)
# imp_channels, field_numbers = refine_chanel_non_temporal(simple_images)
# imp_channels[0].shape

In [262]:
resized_images = resize_with_padding(simple_images)
resized_images[0].shape

(224, 224, 3)

In [263]:
# Load pretrained ResNet model
model = resnet50(pretrained=True)

# Modify the first convolutional layer to accept 6 channels
model.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)

# Remove the final fully connected layer
feature_extractor = nn.Sequential(*list(model.children())[:-1])

/Users/bhumikasadbhave007/tensorflow-test/env/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/bhumikasadbhave007/tensorflow-test/env/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [264]:
images_tensors = [torch.from_numpy(img).float() for img in resized_images]
stacked_images = torch.stack(images_tensors, dim=0)
stacked_images.shape

torch.Size([48, 224, 224, 3])

In [265]:
features = extract_features_cnn(stacked_images, feature_extractor)
print("Features Shape:", features.shape)  # (n, 2048)

/Users/bhumikasadbhave007/Documents/THWS/Semester-4/MASTER-THESIS/Master-Thesis-Github/Master-Thesis/Modeling/model_scripts/feature_extraction.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  images = torch.tensor(images, dtype=torch.float32).permute(0, 3, 1, 2)  # (N, C, H, W)


Features Shape: (48, 2048)


In [266]:
n_clusters = 2  
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(features)

labels = kmeans.labels_

/Users/bhumikasadbhave007/tensorflow-test/env/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [267]:
acc = get_accuracy(field_numbers, labels, config.labels_path)
acc

0.47540983606557374